References:
- https://docs.python.org/3/reference/datamodel.html
- Fluent Python by Luciano Ramalho. Chapter 16: Coroutines

# Coroutine
A coroutine is syntatically like a generator: just a function with the `yield` keyword in its body. But with the `yield` usually appears on the right  side of an expression.

e.g. `datum = yield`


# How Coroutines Evolved from Generators

[PEP 342 – Coroutines via Enhanced Generators](https://www.python.org/dev/peps/pep-0342/)

### `.send() .throw() .close()`

[PEP 380 – Syntax for Delegating to a Subgenerator](https://peps.python.org/pep-0380/)

# Basic Behavior of a Generator Used as a  Coroutine

In [1]:
def simple_coroutine():
    print('-> coroutine started')
    x = yield
    print('-> coroutine received:', x)

In [2]:
my_coro = simple_coroutine()
my_coro

<generator object simple_coroutine at 0x1108b19d0>

In [3]:
next(my_coro)

-> coroutine started


In [4]:
my_coro.send(42)

-> coroutine received: 42


StopIteration: 

In [5]:
import inspect

In [6]:
inspect.getgeneratorstate(my_coro)

'GEN_CLOSED'

In [7]:
my_coro = simple_coroutine()
my_coro.send(1729)

TypeError: can't send non-None value to a just-started generator

In [8]:
def simple_coro2(a):
    print('-> Started: a =', a)
    b = yield a
    print('-> Received: b =', b)
    c = yield a + b
    print('-> Received: c =', c)

In [9]:
my_coro2 = simple_coro2(14)

In [10]:
from inspect import getgeneratorstate
getgeneratorstate (my_coro2)

'GEN_CREATED'

In [11]:
next(my_coro2)

-> Started: a = 14


14

In [12]:
getgeneratorstate(my_coro2)

'GEN_SUSPENDED'

In [13]:
my_coro2.send(28)

-> Received: b = 28


42

In [14]:
my_coro2.send(99)

-> Received: c = 99


StopIteration: 

In [15]:
getgeneratorstate(my_coro2)

'GEN_CLOSED'

#  Coroutine to Compute a Running Average

In [16]:
def averager():
    total = 0.0
    count = 0
    average = None
    while True:  # Infinite loop to  keep  accepting values
        term = yield average  # used to suspend the coroutine
        total += term
        count += 1
        average = total/count

In [17]:
coro_avg = averager()

In [18]:
next(coro_avg)

In [19]:
coro_avg.send(10)

10.0

In [20]:
coro_avg.send(30)

20.0

In [21]:
coro_avg.send(5)

15.0

# Decorators for Coroutine Priming

In [22]:
from functools import wraps

In [23]:
def coroutine(func):
    """Decorator: primes `func` by advancing to first `yield`"""
    @wraps(func)
    def primer(*args,**kwargs):  
        gen = func(*args,**kwargs)  # call the decorated function to get a generator object
        next(gen)  # prime the generator
        return gen  # return it
    return primer

In [24]:
@coroutine  # <5>
def averager():  # <6>
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

In [25]:
coro_avg = averager()

In [26]:
getgeneratorstate(coro_avg)

'GEN_SUSPENDED'

In [27]:
coro_avg.send(10)

10.0

In [28]:
coro_avg.send(30)

20.0

In [29]:
coro_avg.send(5)

15.0

#  Coroutine Termination and Exception Handling

In [30]:
coro_avg.send('spam')

TypeError: unsupported operand type(s) for +=: 'float' and 'str'

In [31]:
class DemoException(Exception):
    """An exception type for the demonstration."""

def demo_exc_handling():
    print('-> coroutine started')
    while True:
        try:
            x = yield
        except DemoException:  # <1>
            print('*** DemoException handled. Continuing...')
        else:  # <2>
            print('-> coroutine received: {!r}'.format(x))
    raise RuntimeError('This line should never run.')

In [32]:
exc_coro = demo_exc_handling()

In [33]:
next(exc_coro)

-> coroutine started


In [34]:
exc_coro.send(11)

-> coroutine received: 11


In [35]:
exc_coro.send(22)

-> coroutine received: 22


In [36]:
exc_coro.close()

In [37]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

In [38]:
exc_coro = demo_exc_handling()

In [39]:
next(exc_coro)

-> coroutine started


In [40]:
exc_coro.send(11)

-> coroutine received: 11


In [41]:
exc_coro.throw(DemoException)

*** DemoException handled. Continuing...


In [42]:
getgeneratorstate(exc_coro)

'GEN_SUSPENDED'

In [43]:
exc_coro = demo_exc_handling()

In [44]:
next(exc_coro)

-> coroutine started


In [45]:
exc_coro.send(11)

-> coroutine received: 11


In [46]:
exc_coro.throw(ZeroDivisionError)

ZeroDivisionError: 

In [47]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

In [48]:
class DemoException(Exception):
    """An exception type for the demonstration."""


def demo_finally():
    print('-> coroutine started')
    try:
        while True:
            try:
                x = yield
            except DemoException:
                print('*** DemoException handled. Continuing...')
            else:
                print('-> coroutine received: {!r}'.format(x))
    finally:
        print('-> coroutine ending')

In [49]:
fin_coro = demo_finally()

In [50]:
next(fin_coro)

-> coroutine started


In [51]:
fin_coro.send(11)

-> coroutine received: 11


In [52]:
fin_coro.close()

-> coroutine ending


In [53]:
fin_coro = demo_finally()

In [54]:
next(fin_coro)

-> coroutine started


In [55]:
fin_coro.send(11)

-> coroutine received: 11


In [56]:
fin_coro.throw(ZeroDivisionError)

-> coroutine ending


ZeroDivisionError: 

`python3 -m doctest coro_exception_demos.py`

# Returning a Value from a Coroutine

In [57]:
from collections import namedtuple

In [58]:
Result = namedtuple('Result', 'count average')

In [59]:
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break  # break loop in order to return a value
        total += term
        count += 1
        average = total/count
    return Result(count, average)

In [60]:
coro_avg = averager()

In [61]:
next(coro_avg)

In [62]:
coro_avg.send(10)

In [63]:
coro_avg.send(30)

In [64]:
coro_avg.send(6.5)

In [65]:
coro_avg.send(None)

StopIteration: Result(count=3, average=15.5)

In [66]:
coro_avg = averager()

In [67]:
next(coro_avg)

In [68]:
coro_avg.send(10)

In [69]:
coro_avg.send(30)

In [70]:
coro_avg.send(6.5)

In [71]:
try:
    coro_avg.send(None)
except StopIteration as exc:
    result = exc.value

In [72]:
result

Result(count=3, average=15.5)

# Using yield from
<img src="delegate.png" width="75%">

In [73]:
def gen():
    for c in 'AB':
        yield c
    for  i in range(1, 3):
        yield i

In [74]:
list(gen())

['A', 'B', 1, 2]

In [75]:
def gen():
    yield from 'AB'
    yield from  range(1, 3)

In [76]:
list(gen())

['A', 'B', 1, 2]

In [77]:
def chain(*iterables):
    for it in iterables:
        yield from it

In [78]:
s = 'ABC'

In [79]:
t = tuple(range(3))

In [80]:
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]